# Читаем файлы PDB

In [112]:
import os
import subprocess
from datetime import datetime
from jinja2 import Template

In [146]:
class Templater:

    def __init__(
            self, 
            path, 
            name,
            vhh_pdb, 
            rbd_pdb, 
            reference_file, 
            ambig_file, 
            run_dir, 
            ncores, 
            mode, 
            ):
        self.path = path
        self.name = name
        self.vhh_pdb = vhh_pdb
        self.rbd_pdb = rbd_pdb
        self.reference_file = reference_file
        self.ambig_file = ambig_file
        self.run_dir = run_dir
        self.ncores = ncores
        self.mode = mode

    def save(self):
        with open("config_template.cfg") as f:
            template = Template(f.read())

        rendered = template.render(
            name=self.name,
            vhh_pdb=self.vhh_pdb,
            rbd_pdb=self.rbd_pdb,
            reference_file=self.reference_file,
            ambig_file=self.ambig_file,
            run_dir=self.run_dir,
            ncores=self.ncores,
            mode=self.mode,
        )

        with open(self.path, "w") as f:
            f.write(rendered)

class RBD(object):

    def __init__(self, file):
        self.path = file
        file_name = self.path.split("/")[-1][:-4].split("_")

        self.variant = file_name[1]
        self.pdb_id = file_name[2]
        self.chain = file_name[3]
        self.name = f"{self.variant}_{self.pdb_id}_{self.chain}"

    def add_active_residues(self, vhh7_active_residues, vhh57_active_residues, vhh58_active_residues):
        self.vhh7_active_residues = vhh7_active_residues
        self.vhh57_active_residues = vhh57_active_residues
        self.vhh58_active_residues = vhh58_active_residues

    def add_passive_residues(self, vhh7_passive_residues, vhh57_passive_residues, vhh58_passive_residues):
        self.vhh7_passive_residues = vhh7_passive_residues
        self.vhh57_passive_residues = vhh57_passive_residues
        self.vhh58_passive_residues = vhh58_passive_residues

    def create_active_passive_string(self):
        active_vhh7 = str(self.vhh7_active_residues).replace(",", "")[1:-1]
        passive_vhh7 = str(self.vhh7_passive_residues).replace(",", "")[1:-1]
        self.active_passive_string_vhh7 = active_vhh7 + "\n" + passive_vhh7
        # self.active_passive_vhh7_path = ""

        active_vhh57 = str(self.vhh57_active_residues).replace(",", "")[1:-1]
        passive_vhh57 = str(self.vhh57_passive_residues).replace(",", "")[1:-1]
        self.active_passive_string_vhh57 = active_vhh57 + "\n" + passive_vhh57
        # self.active_passive_vhh57_path = ""

        active_vhh58 = str(self.vhh58_active_residues).replace(",", "")[1:-1]
        passive_vhh58 = str(self.vhh58_passive_residues).replace(",", "")[1:-1]
        self.active_passive_string_vhh58 = active_vhh58 + "\n" + passive_vhh58
        # self.active_passive_vhh58_path = ""
        self.active_passive_path = {
            "vhh7": "",
            "vhh57": "",
            "vhh58": "",
        }

    def describe(self):
        return (
        "> path: " + self.path + "\n"
        + "variant: " + self.variant + "\n"
        + "pdb_id: " + self.pdb_id + "\n"
        + "chain: " + self.chain + "\n"
        + "vhh7_active_residues: " + str(self.vhh7_active_residues) + "\n"
        + "vhh57_active_residues: " + str(self.vhh57_active_residues) + "\n"
        + "vhh58_active_residues: " + str(self.vhh58_active_residues) + "\n"
        + "vhh7_passive_residues: " + str(self.vhh7_passive_residues) + "\n"
        + "vhh57_passive_residues: " + str(self.vhh57_passive_residues) + "\n"
        + "vhh58_passive_residues: " + str(self.vhh58_passive_residues) + "\n"
    )

class VHH(object):

    def __init__(self, file):
        self.path = file
        file_name = self.path.split("/")[-1].split(".")[0].split("_")

        self.name = file_name[0] + file_name[1]
        self.chain = "H"

    def add_active_residues(self, active_residues):
        self.active_residues = active_residues

    def add_passive_residues(self, passive_residues):
        self.passive_residues = passive_residues

    def create_active_passive_string(self):
        active = str(self.active_residues).replace(",", "")[1:-1]
        passive = str(self.passive_residues).replace(",", "")[1:-1]
        self.active_passive_string = active + "\n" + passive
        self.active_passive_path = ""

    def describe(self):
        return (
            "> path: " + self.path + "\n"
            + "name: " + self.name + "\n"
            + "active_residues: " + str(self.active_residues) + "\n"
            + "passive_residues: " + str(self.passive_residues) + "\n"
        )


class PDBPreparer:

    def __init__(self, antigene_folder, vhh_folder):
        self.antigene_folder = antigene_folder
        self.vhh_folder = vhh_folder
        self.PDBformat = ".pdb"

    def get_files_with_extension(self, folder_path, extension):
        """
        Возвращает список файлов с заданным расширением в указанной папке.

        :param folder_path: Путь к папке (строка)
        :param extension: Расширение файла (например, ".txt", ".py")
        :return: Список путей к файлам
        """
        files = []
        for file in os.listdir(folder_path):
            if file.endswith(extension):
                files.append(os.path.join(folder_path, file))
        return files

    def get_files_lists(self):
        self.antigene_files = self.get_files_with_extension(self.antigene_folder, self.PDBformat)
        self.vhh_files = self.get_files_with_extension(self.vhh_folder, self.PDBformat)

    def parse_rbd_names(self):
        self.get_files_lists()

        self.antigene_objects = list()
        for file in self.antigene_files:
            self.antigene_objects.append(RBD(file=file))

    def parse_vhh_names(self):
        self.get_files_lists()

        self.vhh_objects = list()
        for file in self.vhh_files:
            self.vhh_objects.append(VHH(file=file))

    def add_active_residues_to_vhhs(self):
        active = {
            "vhh7": [27, 28, 29, 30, 35, 36, 37, 38, 56, 57, 58, 59, 62, 64, 63, 65, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117],
            "vhh57": [27, 28, 29, 30, 35, 36, 37, 38, 56, 57, 58, 59, 63, 64, 65, 105, 106, 107, 108, 109, 110, 113, 114, 115, 116, 117],
            "vhh58": [27, 28, 29, 30, 35, 36, 37, 38, 56, 57, 58, 59, 63, 64, 65, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117],
        }

        self.parse_vhh_names()

        for vhh in self.vhh_objects:
            vhh.add_active_residues(active[vhh.name])

    def add_active_residues_to_rbds(self):
        vhh7_active_residues = [369, 370, 372, 373, 374, 375, 376, 377, 378, 379, 384, 385, 407, 408, 410, 411, 431, 432, 433, 434, 435, 436]
        vhh57_active_residues = [416, 417, 418, 419, 420, 421, 422, 423, 424, 455, 456, 457, 458, 459, 460, 473, 475, 489]
        vhh58_active_residues = [353, 355, 380, 381, 394, 395, 396, 397, 408, 411, 412, 413, 414, 423, 424, 425, 426, 427, 428, 429, 430, 462, 463, 464, 465, 514, 515, 516, 519]

        self.parse_rbd_names()

        for rbd in self.antigene_objects:
            rbd.add_active_residues(
                vhh7_active_residues=vhh7_active_residues, 
                vhh57_active_residues=vhh57_active_residues, 
                vhh58_active_residues=vhh58_active_residues,
            )

    def add_active_residues(self):
        self.add_active_residues_to_vhhs()
        self.add_active_residues_to_rbds()

    def get_vhh_passive_residues(self):
        self.add_active_residues_to_vhhs()

        for vhh in self.vhh_objects:
            # Сборка команды
            cmd = [
                "haddock3-restraints",
                "passive_from_active",
                vhh.path,
                str(vhh.active_residues).replace(" ", "")[1:-1]
            ]

            # Запуск команды и захват stdout
            try:
                output = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
                # Поиск строки с числами — берем последнюю строку из вывода с числами
                lines = output.strip().split('\n')
                passive_line = next((line for line in lines[::-1] if all(x.isdigit() or x == ' ' for x in line)), None)

                if passive_line:
                    passive_residues = list(map(int, passive_line.strip().split()))
                    vhh.add_passive_residues(passive_residues)
                else:
                    print(f"Не удалось найти числа в выводе для файла {vhh.path}")

            except subprocess.CalledProcessError as e:
                print(f"Ошибка при выполнении команды для {vhh.path}:\n{e.output}")

    def get_rbd_passive_residues(self):
        self.add_active_residues_to_rbds()

        for rbd in self.antigene_objects:
            # Сборка команды
            cmd_vhh7 = [
                "haddock3-restraints",
                "passive_from_active",
                rbd.path,
                str(rbd.vhh7_active_residues).replace(" ", "")[1:-1]
            ]

            cmd_vhh57 = [
                "haddock3-restraints",
                "passive_from_active",
                rbd.path,
                str(rbd.vhh57_active_residues).replace(" ", "")[1:-1]
            ]

            cmd_vhh58 = [
                "haddock3-restraints",
                "passive_from_active",
                rbd.path,
                str(rbd.vhh58_active_residues).replace(" ", "")[1:-1]
            ]

            # Запуск команды и захват stdout
            try:
                output_vhh7 = subprocess.check_output(cmd_vhh7, stderr=subprocess.STDOUT, text=True)
                output_vhh57 = subprocess.check_output(cmd_vhh57, stderr=subprocess.STDOUT, text=True)
                output_vhh58 = subprocess.check_output(cmd_vhh58, stderr=subprocess.STDOUT, text=True)

                # Поиск строки с числами — берем последнюю строку из вывода с числами
                lines_vhh7 = output_vhh7.strip().split('\n')
                lines_vhh57 = output_vhh57.strip().split('\n')
                lines_vhh58 = output_vhh58.strip().split('\n')

                passive_line_vhh7 = next((line for line in lines_vhh7[::-1] if all(x.isdigit() or x == ' ' for x in line)), None)
                passive_line_vhh57 = next((line for line in lines_vhh57[::-1] if all(x.isdigit() or x == ' ' for x in line)), None)
                passive_line_vhh58 = next((line for line in lines_vhh58[::-1] if all(x.isdigit() or x == ' ' for x in line)), None)

                if passive_line_vhh7:
                    vhh7_passive_residues = list(map(int, passive_line_vhh7.strip().split()))
                if passive_line_vhh57:
                    vhh57_passive_residues = list(map(int, passive_line_vhh57.strip().split()))
                if passive_line_vhh58:
                    vhh58_passive_residues = list(map(int, passive_line_vhh58.strip().split()))
                    
                    rbd.add_passive_residues(vhh7_passive_residues, vhh57_passive_residues, vhh58_passive_residues)
                else:
                    print(f"Не удалось найти числа в выводе для файла {vhh.path}")

            except subprocess.CalledProcessError as e:
                print(f"Ошибка при выполнении команды для {vhh.path}:\n{e.output}")

    def get_passive_residues(self):
        self.get_vhh_passive_residues()
        self.get_rbd_passive_residues()

    def create_active_passive_files(self):
        self.get_vhh_passive_residues()
        for vhh in self.vhh_objects:
            vhh_actpass_path = f"/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/VHH_active-passive/{vhh.name}.act-pass"
            vhh.create_active_passive_string()
            with open(vhh_actpass_path, "w") as file:
                file.write(vhh.active_passive_string)
            vhh.active_passive_path = vhh_actpass_path

        self.get_rbd_passive_residues()
        for rbd in self.antigene_objects:
            rbd.create_active_passive_string()
            
            rbd_vhh7_actpass_path = f"/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/RBD_active-passive/{rbd.name}-vhh7.act-pass"
            with open(rbd_vhh7_actpass_path, "w") as file:
                file.write(rbd.active_passive_string_vhh7)
            rbd.active_passive_path["vhh7"] = rbd_vhh7_actpass_path

            rbd_vhh57_actpass_path = f"/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/RBD_active-passive/{rbd.name}-vhh57.act-pass"
            with open(rbd_vhh57_actpass_path, "w") as file:
                file.write(rbd.active_passive_string_vhh57)
            rbd.active_passive_path["vhh57"] = rbd_vhh57_actpass_path

            rbd_vhh58_actpass_path = f"/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/RBD_active-passive/{rbd.name}-vhh58.act-pass"
            with open(rbd_vhh58_actpass_path, "w") as file:
                file.write(rbd.active_passive_string_vhh58)
            rbd.active_passive_path["vhh58"] = rbd_vhh58_actpass_path

    def run_active_passive_to_ambig(
        self,
        active_passive_one: str,
        active_passive_two: str,
        segid_one: str,
        segid_two: str,
        output_file: str,
    ):
        """
        Запускает haddock3-restraints active_passive_to_ambig с заданными параметрами.

        :param active_passive_one: Путь к первому .act-pass файлу (например, ./restraints/antibody-paratope.act-pass)
        :param active_passive_two: Путь ко второму .act-pass файлу (например, ./restraints/antigen-NMR-epitope.act-pass)
        :param segid_one: Значение --segid-one (например, A)
        :param segid_two: Значение --segid-two (например, B)
        :param output_file: Путь к выходному .tbl файлу (например, ambig-paratope-NMR-epitope.tbl)
        """
        cmd = [
            "haddock3-restraints", "active_passive_to_ambig",
            active_passive_one,
            active_passive_two,
            "--segid-one", segid_one,
            "--segid-two", segid_two
        ]

        try:
            with open(output_file, 'w') as f_out:
                subprocess.run(cmd, stdout=f_out, stderr=subprocess.STDOUT, check=True)
            print(f"Успешно создан файл: {output_file}")
            return output_file
        except subprocess.CalledProcessError as e:
            print(f"Ошибка при выполнении команды: {e}")

    def create_rbd_vhh_ambigs(self):
        self.create_active_passive_files()

        self.ambigs = list()
        self.config_data = list()
        self.ambig_dir = "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/rbd_vhh_ambigs"
        for rbd in self.antigene_objects:
            for vhh in self.vhh_objects:
                self.ambigs.append(self.run_active_passive_to_ambig(
                                        active_passive_one = vhh.active_passive_path,
                                        active_passive_two = rbd.active_passive_path[vhh.name],
                                        segid_one = vhh.chain,
                                        segid_two = rbd.chain,
                                        output_file = f"{self.ambig_dir}/{rbd.name}-{vhh.name}.tbl",
                                    )
                                )
                
                ref_dir = "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/reference_pdbs/vhhs_rbds"
                for ref in os.listdir(ref_dir):
                    ref_path = ref_dir+"/"+ref
                    if rbd.pdb_id in ref:
                        vname = vhh.name[:3] + "_" + vhh.name[3:]
                        if vname in ref:
                            print(vname)
                            dt = datetime.today().strftime("%Y_%m_%d")
                            cfg = Templater(
                                    path=f"/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/configs/{rbd.name}-{vhh.name}-{dt}.cfg",
                                    name=ref,
                                    vhh_pdb=vhh.path, 
                                    rbd_pdb=rbd.path, 
                                    reference_file=ref_path, 
                                    ambig_file=f"{self.ambig_dir}/{rbd.name}-{vhh.name}.tbl", 
                                    run_dir=f"../results/2025_05_11/{rbd.name}-{vhh.name}-{dt}", 
                                    ncores=27, 
                                    mode="local", 
                                )
                            cfg.save()
                            print(f"\t> Успешно создан конфигурационный файл: {cfg.path}")
                            self.config_data.append(cfg)
    
            

In [147]:
antigene_folder = "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/RBDs"
vhh_folder = "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/VHHs"
ext = ".pdb"

parser = PDBPreparer(antigene_folder = antigene_folder, vhh_folder = vhh_folder)
parser.create_rbd_vhh_ambigs()

Успешно создан файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/rbd_vhh_ambigs/WU_6w41_C-vhh58.tbl
vhh_58
	> Успешно создан конфигурационный файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/configs/WU_6w41_C-vhh58-2025_05_11.cfg
Успешно создан файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/rbd_vhh_ambigs/WU_6w41_C-vhh57.tbl
vhh_57
	> Успешно создан конфигурационный файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/configs/WU_6w41_C-vhh57-2025_05_11.cfg
Успешно создан файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/rbd_vhh_ambigs/WU_6w41_C-vhh7.tbl
vhh_7
	> Успешно создан конфигурационный файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/configs/WU_6w41_C-vhh7-2025_05_11.cfg
Успешно создан файл: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/rbd_vhh_ambigs/EG.5.1_8xn2_B-vhh58.tbl
vhh_58
	> Успешно создан конфигурационный файл: /home/mr-re

In [ ]:
class Template:

    def __init__(
            self, 
            vhh_pdb, 
            rbd_pdb, 
            reference_file, 
            ambig_file, 
            run_dir, 
            ncores, 
            mode, 
            ):
        self.vhh_pdb = vhh_pdb
        self.rbd_pdb = rbd_pdb
        self.reference_file = reference_file
        self.ambig_file = ambig_file
        self.run_dir = run_dir
        self.ncores = ncores
        self.mode = mode
        

> path: /home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/RBDs/rbd_WU_6w41_C.pdb
variant: WU
pdb_id: 6w41
chain: C
vhh7_active_residues: [416, 417, 418, 419, 420, 421, 422, 423, 424, 455, 456, 457, 458, 459, 460, 473, 475, 489]
vhh57_active_residues: [416, 417, 418, 419, 420, 421, 422, 423, 424, 455, 456, 457, 458, 459, 460, 473, 475, 489]
vhh58_active_residues: [353, 355, 380, 381, 394, 395, 396, 397, 408, 411, 412, 413, 414, 423, 424, 425, 426, 427, 428, 429, 430, 462, 463, 464, 465, 514, 515, 516, 519]
vhh7_passive_residues: [351, 352, 354, 355, 403, 409, 412, 413, 414, 415, 426, 427, 429, 452, 462, 463, 464, 465, 466, 467, 469, 470, 471, 472, 474, 476, 477, 478, 480, 483, 484, 485, 486, 487, 490, 492, 493, 494]
vhh57_passive_residues: [351, 352, 354, 355, 403, 409, 412, 413, 414, 415, 426, 427, 429, 452, 462, 463, 464, 465, 466, 467, 469, 470, 471, 472, 474, 476, 477, 478, 480, 483, 484, 485, 486, 487, 490, 492, 493, 494]
vhh58_passive_residues: [336, 348, 351, 352, 354,

In [98]:
from jinja2 import Template

# Загрузи шаблон из файла
with open("config_template.cfg") as f:
    template = Template(f.read())

# Данные для подстановки
data = {
    "run_dir": "vhh_rbd_dir",
    "ncores": 26,
    "mode": "local",
    "vhh_pdb": "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/VHHs/vhh_7_by_sabpred_api_new.pdb",
    "rbd_pdb": "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/RBDs/rbd_JN.1_8y18_B.pdb",
    "ambig_file": "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/AMBIGs/rbd_vhh_ambigs/JN.1_8y18_B-vhh7.tbl",
    "reference_file": "/home/mr-red/Рабочий стол/epitope_mapping/docking_11.05.2025/reference_pdbs/rbd_JN.1_8y18_B_vhh_7.pdb",
}

# Рендерим шаблон
config_text = template.render(data)

# Сохраняем в файл
with open("config.cfg", "w") as f:
    f.write(config_text)